In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import re
import emoji
import nltk
import numpy as np
from geopy.geocoders import Nominatim

In [ ]:
data = pd.read_csv('#AbortionRights_2022-10-09.csv')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3618 entries, 0 to 3617
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    3618 non-null   int64 
 1   username      3618 non-null   object
 2   description   3211 non-null   object
 3   location      2452 non-null   object
 4   following     3618 non-null   int64 
 5   followers     3618 non-null   int64 
 6   totaltweets   3618 non-null   int64 
 7   retweetcount  3618 non-null   int64 
 8   text          3618 non-null   object
 9   hashtags      3618 non-null   object
 10  time          3618 non-null   object
dtypes: int64(5), object(6)
memory usage: 311.0+ KB


In [ ]:
nltk.download('words')
dic = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
def cleaner(tweet):
    #Remove @ sign
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) 
    #Remove http links
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)
    tweet = " ".join(tweet.split())
    #Remove Emojis
    tweet_cleaned = ""
    for char in tweet:
      if not emoji.is_emoji(char):
        tweet_cleaned += char
    
    #Remove hashtag sign but keep the text
    tweet_cleaned = tweet_cleaned.replace("#", "").replace("_", " ") .replace("\'", "")
    
    words = []
    for word in nltk.wordpunct_tokenize(tweet_cleaned):
      # if the word in NLTK dictionary or it is a single char
      if word in dic or word.lower() in dic or not word.isalpha():
          words.append(word)

    
    return " ".join(words)

In [ ]:
# clean tweets text data
data['text'] = data['text'].map(lambda x: cleaner(x))

In [ ]:
data.groupby('location').size()

location
                                1
   ’nel quinto coño             1
 Cornfields of NW Indiana       1
 Dallas, TX.                    1
 Kansas - Little 🍎              1
                               ..
🌎seattle, planet earth          1
🏠 Philadelphia, PA (HE/THEY)    1
💗Sapphic                        1
💙🌊🌊💙🐕🦮🐕‍🦺🐈‍⬛🐈☮no DMs            1
🫀                               1
Length: 1271, dtype: int64

In [ ]:
# pre-cleaning of location data
data['location'] = data['location'].replace(np.nan,'')
data['location'] = data['location'].map(lambda x: cleaner(x))

In [ ]:
data["location_lat"]=""
data["location_long"]=""
geolocator = Nominatim(user_agent="myApp")

for i in data.index:
    # if it has location info
    if data.loc[i,'location'] != '':
        #tries fetch address from geopy
        try:
          # use geopy to search for address
          location = geolocator.geocode(data['location'][i])
          # save geo info
          data.loc[i,'location_lat'] = location.latitude
          data.loc[i,'location_long'] = location.longitude

          
          # this runs slow, just to check progress
          print(i)
        except:
          continue

5
7
8
10
13
16
17
18
19
22
23
24
27
28
32
34
46
47
49
50
51
52
54
55
57
60
61
64
66
69
70
71
72
74
75
78
79
80
84
86
90
94
96
97
100
101
103
106
107
108
110
111
115
116
119
120
122
124
125
127
129
131
137
138
139
146
148
149
150
157
159
160
161
164
165
166
170
171
172
174
175
177
178
179
180
181
189
191
192
194
200
201
206
207
208
210
211
212
217
218
221
223
227
229
230
231
236
239
242
248
249
250
252
254
255
256
257
258
259
261
264
266
267
270
272
278
280
281
283
285
287
288
292
293
295
297
303
305
307
313
314
315
316
317
320
322
324
325
328
329
330
333
335
336
337
338
340
341
343
346
348
349
353
354
355
359
361
362
363
374
375
379
381
385
389
392
394
396
403
404
405
406
407
409
410
411
412
413
418
422
423
431
432
435
437
438
439
442
443
451
453
454
457
459
461
463
464
466
468
469
470
473
474
475
476
478
479
480
481
482
483
494
497
501
506
512
513
514
516
517
519
522
527
528
531
533
535
538
548
549
551
552
553
554
556
559
560
563
564
565
566
568
569
570
574
575
576
578
580
581
585
593

In [ ]:
data.in

,Unnamed: 0,username,description,location,following,followers,totaltweets,retweetcount,text,hashtags,time,location_lat,location_long
0,0,lynne_hartman,"Lover of Jesus, my husband of 53 years, our ch...",,336,49,719,40,"Saturday ’ s debate got a little intense , but...",[],2022-10-17 22:56:07+00:00,,
1,1,HarmonyJackson,NaN,,206,141,2663,0,are on the ballot Midterms2022 . Don ’ t f it ...,"['AbortionRights', 'Midterms2022', 'AbortionRi...",2022-10-17 22:40:15+00:00,,
2,2,Drew31435169,#Rigamarole\n#Pureblood,,757,174,27127,40,"Saturday ’ s debate got a little intense , but...",[],2022-10-17 22:40:15+00:00,,
3,3,jamespeipi,NaN,,122,29,192,40,"Saturday ’ s debate got a little intense , but...",[],2022-10-17 22:39:54+00:00,,
4,4,BurnessRaphael,"“When government fears the people, there is li...",,249,27,1697,40,"Saturday ’ s debate got a little intense , but...",[],2022-10-17 22:38:15+00:00,,


In [ ]:
data.to_csv('cleaned.csv')